In [1]:
import pandas as pd
import numpy as np
import copy
import csv
from collections import defaultdict


from tqdm import tqdm
from Bio import SeqIO, SearchIO

In [11]:
df_cyt = pd.read_csv('/home/marfa/Desktop/lab/mycofactocin/mftb/cluster_res/new_mftb_cluster_rep_seq_node.csv', sep = ',')
df_cyt['seq_name'] = df_cyt['Description'].str.split(' ')
df_cyt['seq_name'] = [x[0] for x in df_cyt['seq_name']]
df_cyt.index = df_cyt['seq_name'].copy()
del df_cyt['seq_name']
df_cyt

,Description,List of IDs in Rep Node,name,Number of IDs in Rep Node,selected,Sequence,Sequence Length,Sequence Source,shared name
seq_name,,,,,,,,,
WP_239101810.1,WP_239101810.1 mycofactocin radical SAM matura...,zzzz657,zzzz657,1,False,MTDPLDEAWELSPRVALRPEPFGALAYHFGNRRLTFLKRPELVSVL...,557,None,zzzz657
WP_228638331.1,WP_228638331.1 mycofactocin radical SAM matura...,zzzz574,zzzz574,1,False,MTNPLDEAWELSPSVALRPEPFGALAYHFGNRRLTFLKRRELVAVV...,529,None,zzzz574
WP_189138050.1,WP_189138050.1 mycofactocin radical SAM matura...,zzzzz54,zzzzz54,1,False,MTSPLDQAWELSPSVALRPETFGAMAYHFGNRRLTFLKRTELVSVV...,525,None,zzzzz54
WP_255280863.1,WP_255280863.1 mycofactocin radical SAM matura...,zzzz622,zzzz622,1,False,MARTPAASDFDLDAAWDLHPQVAVRPEPFGALLYHYGTRRLSFLKD...,523,None,zzzz622
WP_081878369.1,WP_081878369.1 mycofactocin radical SAM matura...,zzzz154,zzzz154,1,False,MFDLDDRWQLHPRVALRPEPFGALLYHFGTRKLSFLKSPRIVEIVR...,518,None,zzzz154
...,...,...,...,...,...,...,...,...,...
WP_257865691.1,WP_257865691.1 mycofactocin biosynthesis chape...,zzzz347,zzzz347,1,False,HFGTRKLSFLKNLTVVDIVKSLADHDSAEAALLAAGVTPAQRPLYV...,63,None,zzzz347
WP_214290027.1,WP_214290027.1 mycofactocin biosynthesis chape...,zzzz141,zzzz141,1,False,MWGLLTVPAPAQARRADSSEFDPDRGWRLHPQVAVRPEPFGALLYH...,58,None,zzzz141
WP_119595456.1,WP_119595456.1 mycofactocin biosynthesis chape...,zzzz167,zzzz167,1,False,MSAMIDEAPAGFDWTRPWQLHPQVSLRPEPFGALLYHFGTRKLSFL...,58,None,zzzz167


In [3]:
rodeo_all = pd.read_csv('/home/marfa/Desktop/lab/mycofactocin/mftb/mftb_psi_rodeo/main_co_occur_rodeo_psi.csv', sep = ",")
rodeo = rodeo_all.loc[:,['Query','Protein_acc']]
rodeo

,Query,Protein_acc
0,WP_145853129.1,WP_187365872.1
1,WP_145853129.1,WP_187365873.1
2,WP_145853129.1,WP_145853125.1
3,WP_145853129.1,WP_145853126.1
4,WP_145853129.1,WP_145853127.1
...,...,...
11152,WP_015759610.1,WP_015759614.1
11153,WP_015759610.1,WP_015759615.1
11154,WP_015759610.1,WP_015759616.1
11155,WP_015759610.1,WP_015759617.1


In [4]:
file = '/home/marfa/Desktop/lab/mycofactocin/mftb/hmm_for_domains/parsed_hmm_per_dom_tbl.txt'
res_domains = dict()
x = 10
count = 0
for res in SearchIO.parse(file, 'hmmscan3-domtab'):
        res_domains[res.id] = {}
        for hit in res.hits:
            if hit.id[:4] == 'TIGR':
                    call = hit.description.split(":")[0]
            else:
                    call = hit.id
            for hsp in hit.hsps:
                overlap = False
                if res_domains[res.id].keys(): #not empty
                    names = copy.deepcopy(list(res_domains[res.id].keys()))
                    for name in names: #проверка на перекрытие с раннее добавленными генами 
                        for q_range in res_domains[res.id][name]:
                            if min(hsp.env_end, q_range[1]) - max(hsp.env_start, q_range[0]) > x:
                                overlap = True
                                break
                        if overlap:
                            break
                    if not overlap: 
                        if call not in res_domains[res.id]:
                            res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                            
                        if (hsp.env_start, hsp.env_end) not in res_domains[res.id][call]:
                                        #для случая мультидоменного белка с одинаковыми доменами: 
                                        #один и тот же белок выравнивается на несколько мест
                            res_domains[res.id][call].append((hsp.env_start, hsp.env_end))
                       
                else:
                    res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]

In [5]:
for ke, value in res_domains.items(): #сортирую по началу выравнивания гена на регион
    res_domains[ke] = {k: v for k, v in sorted(res_domains[ke].items(), key=lambda item: item[1][0][0])}


domains_names = {}
for key, value in res_domains.items():
    domains_names[key] = list(value.keys())

domains_names = pd.DataFrame({'domains': domains_names.values()}, index =res_domains.keys())
domains_names

,domains
WP_011566059.1,[MnhB]
WP_011566060.1,[2a6301s01]
WP_011566061.1,"[Methyltransf_23, Methyltransf_12, mycofact_gl..."
WP_011566062.1,[Creatininase]
WP_011566063.1,[actino_HemFlav]
...,...
WP_160900598.1,"[DoxX_3, DoxX]"
WP_160901386.1,[DNA_alkylation]
WP_160900597.1,"[Pyr_redox_2, Reductase_C]"
WP_160900596.1,"[FAM70, DUF3040]"


In [6]:
domains_region = {k:[] for k in rodeo['Query'].unique()}

for query in tqdm(domains_region.keys()):
    temp = rodeo[rodeo['Query'] == query]
    for prot_id in temp['Protein_acc']:
        if prot_id in domains_names.index:
            for domains in domains_names.loc[prot_id]:
                domains_region[query] += domains

                
dom_region = pd.DataFrame({'domains': domains_region.values()} , index = domains_region.keys())
dom_region

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 681/681 [00:01<00:00, 648.00it/s]


,domains
WP_145853129.1,"[STAS_2, TetR_N, TetR_C_1, CSD, pcrA, OsmC, my..."
WP_056154608.1,"[efflux_EmrB, DUF6286, mycofact_glyco, SDR_sub..."
WP_111258449.1,"[PemK_toxin, HicB, Arc, HTH_11, HTH_PafC, WYL,..."
WP_212516844.1,"[3oxo_ACP_reduc, kduD, pyruv_carbox, GntR, FCD..."
WP_040718660.1,"[NDMA_methanol, AAA_5, 26Sp45, Sigma54_activ_2..."
...,...
WP_212844265.1,"[UreD, salicylate_mono, TetR_C_31, Laminin_G_3..."
WP_050670291.1,"[2TM, DUF779, Aldedh, actino_HemFlav, mycofact..."
WP_122191250.1,"[Aldedh, PA_CoA_ligase, Rxyl_3153, adh_short_C..."
WP_011208766.1,"[VWA_CoxE, meth_Rta_06860, Methyltransf_25, PA..."


In [12]:
df_cyt = df_cyt.join(dom_region)
df_cyt

,Description,List of IDs in Rep Node,name,Number of IDs in Rep Node,selected,Sequence,Sequence Length,Sequence Source,shared name,domains
seq_name,,,,,,,,,,
WP_239101810.1,WP_239101810.1 mycofactocin radical SAM matura...,zzzz657,zzzz657,1,False,MTDPLDEAWELSPRVALRPEPFGALAYHFGNRRLTFLKRPELVSVL...,557,None,zzzz657,NaN
WP_228638331.1,WP_228638331.1 mycofactocin radical SAM matura...,zzzz574,zzzz574,1,False,MTNPLDEAWELSPSVALRPEPFGALAYHFGNRRLTFLKRRELVAVV...,529,None,zzzz574,"[mycofact_glyco, SDR_subfam_2, mycofact_creat,..."
WP_189138050.1,WP_189138050.1 mycofactocin radical SAM matura...,zzzzz54,zzzzz54,1,False,MTSPLDQAWELSPSVALRPETFGAMAYHFGNRRLTFLKRTELVSVV...,525,None,zzzzz54,"[3a0107s01c2, 3a0107s02c, phosphate_pstC, ptsS..."
WP_255280863.1,WP_255280863.1 mycofactocin radical SAM matura...,zzzz622,zzzz622,1,False,MARTPAASDFDLDAAWDLHPQVAVRPEPFGALLYHYGTRRLSFLKD...,523,None,zzzz622,NaN
WP_081878369.1,WP_081878369.1 mycofactocin radical SAM matura...,zzzz154,zzzz154,1,False,MFDLDDRWQLHPRVALRPEPFGALLYHFGTRKLSFLKSPRIVEIVR...,518,None,zzzz154,"[decarb_PcaC, DJ-1_PfpI, Aldo_ket_red, HTH_3, ..."
...,...,...,...,...,...,...,...,...,...,...
WP_257865691.1,WP_257865691.1 mycofactocin biosynthesis chape...,zzzz347,zzzz347,1,False,HFGTRKLSFLKNLTVVDIVKSLADHDSAEAALLAAGVTPAQRPLYV...,63,None,zzzz347,NaN
WP_214290027.1,WP_214290027.1 mycofactocin biosynthesis chape...,zzzz141,zzzz141,1,False,MWGLLTVPAPAQARRADSSEFDPDRGWRLHPQVAVRPEPFGALLYH...,58,None,zzzz141,"[Glyoxalase_6, DUF2332, mycofact_TetR, mycofac..."
WP_119595456.1,WP_119595456.1 mycofactocin biosynthesis chape...,zzzz167,zzzz167,1,False,MSAMIDEAPAGFDWTRPWQLHPQVSLRPEPFGALLYHFGTRKLSFL...,58,None,zzzz167,"[mycofact_TetR, mycofactocin, Mycofactocin_RRE]"


In [13]:
rodeo_all = rodeo_all.loc[:,['Query','Protein_acc', 'start','end', 'dir', 'PfamID1', 'Name1', 'Description1']]
rodeo_all

,Query,Protein_acc,start,end,dir,PfamID1,Name1,Description1
0,WP_145853129.1,WP_187365872.1,891146,890102,-,NaN,NaN,NaN
1,WP_145853129.1,WP_187365873.1,891477,891165,-,TIGR00377,TIGR00377,ant_ant_sig: anti-anti-sigma factor
2,WP_145853129.1,WP_145853125.1,891619,892345,+,PF02909,TetR_C_1,"Tetracyclin repressor-like, C-terminal domain"
3,WP_145853129.1,WP_145853126.1,892572,892368,-,PF00313,CSD,'Cold-shock' DNA-binding domain
4,WP_145853129.1,WP_145853127.1,892834,892603,-,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11152,WP_015759610.1,WP_015759614.1,4443259,4442230,-,PF00766,ETF_alpha,Electron transfer flavoprotein FAD-binding domain
11153,WP_015759610.1,WP_015759615.1,4445244,4443258,-,TIGR03997,TIGR03997,mycofact_OYE_2: mycofactocin system FadH/OYE f...
11154,WP_015759610.1,WP_015759616.1,4447200,4445256,-,TIGR03997,TIGR03997,mycofact_OYE_2: mycofactocin system FadH/OYE f...
11155,WP_015759610.1,WP_015759617.1,4447352,4447226,-,TIGR03969,TIGR03969,mycofactocin: mycofactocin precursor


In [14]:
res = pd.DataFrame(data = np.array([0]*8).reshape((1,8)), index = [0,], columns = ['Query', 'Protein_acc', 'start', 'end', 'dir', 'PfamID1', 'Name1','Description1'])
query = rodeo_all.loc[:,'Query'].drop_duplicates().array
rodeo_all = rodeo_all.iloc[:,:8]
for que in query:
    sign = rodeo_all[rodeo_all['Protein_acc'] == que]['dir'].values #знак нити на кот. лежит mftB
    buffer = rodeo_all[rodeo_all['Query'] == que]
    buffer = buffer[buffer['dir'] == sign[0]] #сохраняем только те элементы которые лежат на одной с mftB нити
    if buffer.shape[0] > 1: #рассматриваем кластеры состоящие из более чем одного гена(mftB)
        buffer['start_min'] = buffer.loc[:, ['start', 'end']].min(axis = 1) #делаем расстояния неориентированными
        buffer['end_max'] = buffer.loc[:, ['start', 'end']].max(axis = 1)
        res = pd.concat([res, buffer], ignore_index = True)

res = res[1:] #первая строчка состояла из 0
res

,Query,Protein_acc,start,end,dir,PfamID1,Name1,Description1,start_min,end_max
1,WP_145853129.1,WP_187365872.1,891146,890102,-,NaN,NaN,NaN,890102.0,891146.0
2,WP_145853129.1,WP_187365873.1,891477,891165,-,TIGR00377,TIGR00377,ant_ant_sig: anti-anti-sigma factor,891165.0,891477.0
3,WP_145853129.1,WP_145853126.1,892572,892368,-,PF00313,CSD,'Cold-shock' DNA-binding domain,892368.0,892572.0
4,WP_145853129.1,WP_145853127.1,892834,892603,-,NaN,NaN,NaN,892603.0,892834.0
5,WP_145853129.1,WP_246115648.1,894020,893531,-,PF02566,OsmC,OsmC-like protein,893531.0,894020.0
...,...,...,...,...,...,...,...,...,...,...
7586,WP_015759610.1,WP_015759614.1,4443259,4442230,-,PF00766,ETF_alpha,Electron transfer flavoprotein FAD-binding domain,4442230.0,4443259.0
7587,WP_015759610.1,WP_015759615.1,4445244,4443258,-,TIGR03997,TIGR03997,mycofact_OYE_2: mycofactocin system FadH/OYE f...,4443258.0,4445244.0
7588,WP_015759610.1,WP_015759616.1,4447200,4445256,-,TIGR03997,TIGR03997,mycofact_OYE_2: mycofactocin system FadH/OYE f...,4445256.0,4447200.0
7589,WP_015759610.1,WP_015759617.1,4447352,4447226,-,TIGR03969,TIGR03969,mycofactocin: mycofactocin precursor,4447226.0,4447352.0


In [15]:
def check_neighbours(df, que):
        names = []
        buffer = df[df['Query'] == que]
        buffer.index = range(buffer.shape[0])
        mftB = buffer[buffer['Protein_acc'] == que]
        before = buffer.iloc[:mftB.index[0]+1].sort_values(by = 'start_min', ascending=False)
        before.index = range(before.shape[0])
        after = buffer.iloc[mftB.index[0]:]

        for i, row in before.iterrows():
            if (i > before.index[0]) and (buffer.iloc[i-1]['start_min'] - row['end_max'] > x):
                before = before.drop(list(range(i-1,before.shape[0])))
                break
                
        #надо как-то учесть перекрывание генов: если перекрываются, либо сразу выкидывать оперон, 
        #либо выкидывать ген, который идет внахелст
        
        for i, row in after.iterrows():
            if (i > after.index[0]) and (row['start_min'] - buffer.iloc[i-1]['end_max'] > x):
                after = after.drop(list(range(i-1, after.shape[0])))
                break
        buffer = pd.concat([before, after],axis = 0).drop_duplicates().sort_values(by = 'start_min')
        buffer.index = range(buffer.shape[0])
        return buffer

In [16]:
x = 100
counts = pd.DataFrame(data = np.array([0]*8).reshape((1,8)), index = [0,], columns = ['Query', 'Protein_acc', 'start', 'end', 'dir', 'PfamID1', 'Name1','Description1'])

for que in res['Query'].drop_duplicates():
    checked_que = check_neighbours(res,que)
    if checked_que.shape[0] > 1:
        counts = pd.concat([counts, checked_que], ignore_index=True)
counts = counts[1:].dropna(how='all',subset = ['Name1']).drop_duplicates()
counts

,Query,Protein_acc,start,end,dir,PfamID1,Name1,Description1,start_min,end_max
1,WP_145853129.1,WP_246115648.1,894020,893531,-,PF02566,OsmC,OsmC-like protein,893531.0,894020.0
2,WP_145853129.1,WP_246115687.1,895302,894114,-,TIGR03965,TIGR03965,mycofact_glyco: mycofactocin system glycosyltr...,894114.0,895302.0
3,WP_145853129.1,WP_145853128.1,896959,895654,-,TIGR03962,TIGR03962,mycofact_rSAM: mycofactocin radical SAM maturase,895654.0,896959.0
4,WP_145853129.1,WP_145853129.1,897230,896942,-,RREFam016,Mycofactocin_RRE,Stand-alone RRE protein in a mycofactocin cluster,896942.0,897230.0
5,WP_145853129.1,WP_145853130.1,897360,897252,-,TIGR03969,TIGR03969,mycofactocin: mycofactocin precursor,897252.0,897360.0
...,...,...,...,...,...,...,...,...,...,...
5243,WP_015759610.1,WP_015759614.1,4443259,4442230,-,PF00766,ETF_alpha,Electron transfer flavoprotein FAD-binding domain,4442230.0,4443259.0
5244,WP_015759610.1,WP_015759615.1,4445244,4443258,-,TIGR03997,TIGR03997,mycofact_OYE_2: mycofactocin system FadH/OYE f...,4443258.0,4445244.0
5245,WP_015759610.1,WP_015759616.1,4447200,4445256,-,TIGR03997,TIGR03997,mycofact_OYE_2: mycofactocin system FadH/OYE f...,4445256.0,4447200.0
5246,WP_015759610.1,WP_015759617.1,4447352,4447226,-,TIGR03969,TIGR03969,mycofactocin: mycofactocin precursor,4447226.0,4447352.0


In [17]:
name = []
for i,row in counts.iterrows():
    if row['Name1'][:4]  == 'TIGR':
        name.append(row['Description1'].split(":")[0])
    else: 
        name.append(row['Name1'])
counts['Name'] = name

del counts['start']
del counts['start_min']
del counts['end']
del counts['end_max']
del counts['Name1']
del counts['Description1']

In [18]:
dict_operon  = defaultdict(list)

for que in counts['Query'].drop_duplicates():
    dict_operon[que].extend(counts[counts['Query'] == que]['Name'])

dict_operon = pd.DataFrame({'operon':dict_operon.values()}, index = dict_operon.keys())
dict_operon

,operon
WP_145853129.1,"[OsmC, mycofact_glyco, mycofact_rSAM, Mycofact..."
WP_056154608.1,"[actino_HemFlav, mycofact_rSAM, Mycofactocin_R..."
WP_111258449.1,"[mycofactocin, Mycofactocin_RRE, mycofact_rSAM..."
WP_212516844.1,"[FCD, P450_rel_GT_act, mycofactocin, Mycofacto..."
WP_040718660.1,"[actino_HemFlav, mycofact_rSAM, Mycofactocin_R..."
...,...
WP_212844265.1,"[mycofactocin, Mycofactocin_RRE, mycofact_rSAM..."
WP_050670291.1,"[SDR_subfam_2, mycofact_creat, mycofact_rSAM, ..."
WP_122191250.1,"[mycofactocin, Mycofactocin_RRE, mycofact_rSAM..."
WP_011208766.1,"[actino_HemFlav, mycofact_rSAM, Mycofactocin_R..."


In [19]:
df_cyt = df_cyt.join(dict_operon)
df_cyt

,Description,List of IDs in Rep Node,name,Number of IDs in Rep Node,selected,Sequence,Sequence Length,Sequence Source,shared name,domains,operon
seq_name,,,,,,,,,,,
WP_239101810.1,WP_239101810.1 mycofactocin radical SAM matura...,zzzz657,zzzz657,1,False,MTDPLDEAWELSPRVALRPEPFGALAYHFGNRRLTFLKRPELVSVL...,557,None,zzzz657,NaN,NaN
WP_228638331.1,WP_228638331.1 mycofactocin radical SAM matura...,zzzz574,zzzz574,1,False,MTNPLDEAWELSPSVALRPEPFGALAYHFGNRRLTFLKRRELVAVV...,529,None,zzzz574,"[mycofact_glyco, SDR_subfam_2, mycofact_creat,...","[mycofact_creat, mycofact_rSAM, mycofactocin, ..."
WP_189138050.1,WP_189138050.1 mycofactocin radical SAM matura...,zzzzz54,zzzzz54,1,False,MTSPLDQAWELSPSVALRPETFGAMAYHFGNRRLTFLKRTELVSVV...,525,None,zzzzz54,"[3a0107s01c2, 3a0107s02c, phosphate_pstC, ptsS...","[mycofact_rSAM, actino_HemFlav, mycofact_creat..."
WP_255280863.1,WP_255280863.1 mycofactocin radical SAM matura...,zzzz622,zzzz622,1,False,MARTPAASDFDLDAAWDLHPQVAVRPEPFGALLYHYGTRRLSFLKD...,523,None,zzzz622,NaN,NaN
WP_081878369.1,WP_081878369.1 mycofactocin radical SAM matura...,zzzz154,zzzz154,1,False,MFDLDDRWQLHPRVALRPEPFGALLYHFGTRKLSFLKSPRIVEIVR...,518,None,zzzz154,"[decarb_PcaC, DJ-1_PfpI, Aldo_ket_red, HTH_3, ...","[mycofact_glyco, mycofact_rSAM, mycofactocin, ..."
...,...,...,...,...,...,...,...,...,...,...,...
WP_257865691.1,WP_257865691.1 mycofactocin biosynthesis chape...,zzzz347,zzzz347,1,False,HFGTRKLSFLKNLTVVDIVKSLADHDSAEAALLAAGVTPAQRPLYV...,63,None,zzzz347,NaN,NaN
WP_214290027.1,WP_214290027.1 mycofactocin biosynthesis chape...,zzzz141,zzzz141,1,False,MWGLLTVPAPAQARRADSSEFDPDRGWRLHPQVAVRPEPFGALLYH...,58,None,zzzz141,"[Glyoxalase_6, DUF2332, mycofact_TetR, mycofac...","[mycofactocin, Mycofactocin_RRE]"
WP_119595456.1,WP_119595456.1 mycofactocin biosynthesis chape...,zzzz167,zzzz167,1,False,MSAMIDEAPAGFDWTRPWQLHPQVSLRPEPFGALLYHFGTRKLSFL...,58,None,zzzz167,"[mycofact_TetR, mycofactocin, Mycofactocin_RRE]","[mycofactocin, Mycofactocin_RRE]"


__Таблица df получена из cytoscape по результатам efi-est из mftb psi-blast'a. По этому же blast'у запускался hmmsscan, как видим для некоторых белков так и не нашлись домены в соотвествующих геномных регионах__ 

In [ ]:
# no_annot = pd.read_csv('/home/marfa/Desktop/lab/mycofactocin/mftb/hmm_for_domains/count/mftb_no_domain_counts_per_region.csv',index_col=0, nrows = 11)
# no_annot

In [21]:
dict_add = dict()
with open('domains_arch_regions.csv', newline='') as csvfile3:
    file = csv.reader(csvfile3, delimiter=',')
    for i, row in enumerate(file):
        if i == 25:
            break
        elif i>0:
            dict_add[row[0]] = [0]
dict_add = pd.DataFrame(dict_add, index = df_cyt.index)
dict_add

,mycofactocin,Mycofactocin_RRE,mycofact_rSAM,actino_HemFlav,mycofact_glyco,mycofact_creat,mycofact_TetR,SDR_subfam_1,Rv0697,SDR_subfam_2,...,VWA_CoxE,mycofact_OYE_1,Fe-ADH,efflux_EmrB,Rxyl_3153,Response_reg~GerE,CoA_transf_3,Glyoxalase_6,Sig70_famx2,DNA_alkylation
seq_name,,,,,,,,,,,,,,,,,,,,,
WP_239101810.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_228638331.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_189138050.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_255280863.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_081878369.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_257865691.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_214290027.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_119595456.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_to_color = df_cyt.join(pd.DataFrame(dict_add))
df_to_color

,Description,List of IDs in Rep Node,name,Number of IDs in Rep Node,selected,Sequence,Sequence Length,Sequence Source,shared name,domains,...,VWA_CoxE,mycofact_OYE_1,Fe-ADH,efflux_EmrB,Rxyl_3153,Response_reg~GerE,CoA_transf_3,Glyoxalase_6,Sig70_famx2,DNA_alkylation
seq_name,,,,,,,,,,,,,,,,,,,,,
WP_239101810.1,WP_239101810.1 mycofactocin radical SAM matura...,zzzz657,zzzz657,1,False,MTDPLDEAWELSPRVALRPEPFGALAYHFGNRRLTFLKRPELVSVL...,557,None,zzzz657,NaN,...,0,0,0,0,0,0,0,0,0,0
WP_228638331.1,WP_228638331.1 mycofactocin radical SAM matura...,zzzz574,zzzz574,1,False,MTNPLDEAWELSPSVALRPEPFGALAYHFGNRRLTFLKRRELVAVV...,529,None,zzzz574,"[mycofact_glyco, SDR_subfam_2, mycofact_creat,...",...,0,0,0,0,0,0,0,0,0,0
WP_189138050.1,WP_189138050.1 mycofactocin radical SAM matura...,zzzzz54,zzzzz54,1,False,MTSPLDQAWELSPSVALRPETFGAMAYHFGNRRLTFLKRTELVSVV...,525,None,zzzzz54,"[3a0107s01c2, 3a0107s02c, phosphate_pstC, ptsS...",...,0,0,0,0,0,0,0,0,0,0
WP_255280863.1,WP_255280863.1 mycofactocin radical SAM matura...,zzzz622,zzzz622,1,False,MARTPAASDFDLDAAWDLHPQVAVRPEPFGALLYHYGTRRLSFLKD...,523,None,zzzz622,NaN,...,0,0,0,0,0,0,0,0,0,0
WP_081878369.1,WP_081878369.1 mycofactocin radical SAM matura...,zzzz154,zzzz154,1,False,MFDLDDRWQLHPRVALRPEPFGALLYHFGTRKLSFLKSPRIVEIVR...,518,None,zzzz154,"[decarb_PcaC, DJ-1_PfpI, Aldo_ket_red, HTH_3, ...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_257865691.1,WP_257865691.1 mycofactocin biosynthesis chape...,zzzz347,zzzz347,1,False,HFGTRKLSFLKNLTVVDIVKSLADHDSAEAALLAAGVTPAQRPLYV...,63,None,zzzz347,NaN,...,0,0,0,0,0,0,0,0,0,0
WP_214290027.1,WP_214290027.1 mycofactocin biosynthesis chape...,zzzz141,zzzz141,1,False,MWGLLTVPAPAQARRADSSEFDPDRGWRLHPQVAVRPEPFGALLYH...,58,None,zzzz141,"[Glyoxalase_6, DUF2332, mycofact_TetR, mycofac...",...,0,0,0,0,0,0,0,0,0,0
WP_119595456.1,WP_119595456.1 mycofactocin biosynthesis chape...,zzzz167,zzzz167,1,False,MSAMIDEAPAGFDWTRPWQLHPQVSLRPEPFGALLYHFGTRKLSFL...,58,None,zzzz167,"[mycofact_TetR, mycofactocin, Mycofactocin_RRE]",...,0,0,0,0,0,0,0,0,0,0


In [23]:
for mftb in df_to_color.index:
    for prot in df_to_color.columns[10:]:
        if type(df_to_color.loc[(mftb, 'domains')]) != float :  #to check that 'domains' is not NaN
            if prot in df_to_color.loc[(mftb, 'domains')]:
                df_to_color.loc[(mftb, prot)] += 1
            elif prot == mftb: #если это белок без аннотации и доменов
                df_to_color.loc[(mftb, prot)] += 1

df_to_color

,Description,List of IDs in Rep Node,name,Number of IDs in Rep Node,selected,Sequence,Sequence Length,Sequence Source,shared name,domains,...,VWA_CoxE,mycofact_OYE_1,Fe-ADH,efflux_EmrB,Rxyl_3153,Response_reg~GerE,CoA_transf_3,Glyoxalase_6,Sig70_famx2,DNA_alkylation
seq_name,,,,,,,,,,,,,,,,,,,,,
WP_239101810.1,WP_239101810.1 mycofactocin radical SAM matura...,zzzz657,zzzz657,1,False,MTDPLDEAWELSPRVALRPEPFGALAYHFGNRRLTFLKRPELVSVL...,557,None,zzzz657,NaN,...,0,0,0,0,0,0,0,0,0,0
WP_228638331.1,WP_228638331.1 mycofactocin radical SAM matura...,zzzz574,zzzz574,1,False,MTNPLDEAWELSPSVALRPEPFGALAYHFGNRRLTFLKRRELVAVV...,529,None,zzzz574,"[mycofact_glyco, SDR_subfam_2, mycofact_creat,...",...,0,0,0,0,0,0,1,0,0,0
WP_189138050.1,WP_189138050.1 mycofactocin radical SAM matura...,zzzzz54,zzzzz54,1,False,MTSPLDQAWELSPSVALRPETFGAMAYHFGNRRLTFLKRTELVSVV...,525,None,zzzzz54,"[3a0107s01c2, 3a0107s02c, phosphate_pstC, ptsS...",...,0,0,0,0,1,0,0,0,0,0
WP_255280863.1,WP_255280863.1 mycofactocin radical SAM matura...,zzzz622,zzzz622,1,False,MARTPAASDFDLDAAWDLHPQVAVRPEPFGALLYHYGTRRLSFLKD...,523,None,zzzz622,NaN,...,0,0,0,0,0,0,0,0,0,0
WP_081878369.1,WP_081878369.1 mycofactocin radical SAM matura...,zzzz154,zzzz154,1,False,MFDLDDRWQLHPRVALRPEPFGALLYHFGTRKLSFLKSPRIVEIVR...,518,None,zzzz154,"[decarb_PcaC, DJ-1_PfpI, Aldo_ket_red, HTH_3, ...",...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_257865691.1,WP_257865691.1 mycofactocin biosynthesis chape...,zzzz347,zzzz347,1,False,HFGTRKLSFLKNLTVVDIVKSLADHDSAEAALLAAGVTPAQRPLYV...,63,None,zzzz347,NaN,...,0,0,0,0,0,0,0,0,0,0
WP_214290027.1,WP_214290027.1 mycofactocin biosynthesis chape...,zzzz141,zzzz141,1,False,MWGLLTVPAPAQARRADSSEFDPDRGWRLHPQVAVRPEPFGALLYH...,58,None,zzzz141,"[Glyoxalase_6, DUF2332, mycofact_TetR, mycofac...",...,0,0,0,0,0,0,0,1,0,0
WP_119595456.1,WP_119595456.1 mycofactocin biosynthesis chape...,zzzz167,zzzz167,1,False,MSAMIDEAPAGFDWTRPWQLHPQVSLRPEPFGALLYHFGTRKLSFL...,58,None,zzzz167,"[mycofact_TetR, mycofactocin, Mycofactocin_RRE]",...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#df_to_color.to_csv('/home/marfa/Desktop/lab/mycofactocin/mftb/cluster_res/new_to_cyt_mftb_cluster_rep_seq_node_with_operon.csv')

In [ ]:
blast_for_native = pd.read_csv('/home/marfa/Desktop/lab/mycofactocin/mftb/cluster_res/mftb_seq_id_for_blast.csv', index_col = 'seq_name', sep = ',')
blast_for_native

In [ ]:
# with open('/home/marfa/Desktop/lab/mycofactocin/mftb/cluster_res/mftb_seq_id_for_blast.fa', 'w') as file:
#     for name in blast_for_native.index:
#         x = blast_for_native.loc[name,'Sequence']
#         file.write('>' + name + '\n')
#         file.write(x + '\n')

In [ ]:
# a = ['mycofact_glyco', 'SDR_subfam_2', 'mycofact_MftB', 'mycofact_creat', 'mycofact_rSAM', 'mycofactocin', 'ECH_2', 'PaaB1', 'CoA_transf_3', 'Amidohydro_3', 'ECH_1', 'DUF2889', 'VP_N-CPKC']
# x = {'mycofactocin~mycofact_glyco': 3}
# for key in x.keys():
#     size = len(key.split('~'))
#     for subset in list(itertools.permutations(a, size)):
#         if key in '~'.join(subset):
#             print('yes')

In [ ]:
# dom_known = pd.DataFrame({'mycofactocin':'MftA'
#             'Mycofactocin_RRE'+'~'+ 'mycofact_MftB': 'MftB',
#             'mycofact_rSAM': 'MftC',
#             'mycofact_TetR'+'~'+'TetR_N': 'MftR',
#             'actino_HemFlav'+'~'+'Rv0694': 'MftD',
#             'mycofact_glyco' : 'MftF',
#             'mycofact_creat':'creat'})

In [24]:
coloring = {mftb:0 for mftb in df_to_color.index}

for mftb in df_to_color.index:
    buf = df_to_color.loc[mftb, :]
    count = 0
    if mftb == 'WP_082966316.1': #closest to original
        coloring[mftb] = 888 
        continue
    if type(buf['domains']) != float:
        if 'mycofactocin' in buf['domains'] and ('Mycofactocin_RRE' in buf['domains'] or 'mycofact_MftB' in buf['domains']) and 'mycofact_rSAM' in buf['domains']: 
            #min cluster
            count = 1
            if 'mycofact_TetR' in buf['domains']: #MftR and min
                count = 2
                if 'mycofact_glyco' in buf['domains']: #MftF and MftR and min
                    count = 5
                    if 'actino_HemFlav' in buf['domains']: #MftD and MftF and MftR and min
                        count = 24
                        if 'Rv0697' in buf['domains']: #Rv0697 and MftD and MftF and MftR and min
                            count = 21

                    elif 'Rv0697' in buf['domains']: #Rv0697 and MftF and MftR and min
                        count = 25

                elif 'actino_HemFlav' in buf['domains']: #MftD and MftR and min
                    count == 11
                    if 'Rv0697' in buf['domains']: #Rv0697 and MftD and MftR and min
                        count = 17

                elif 'Rv0697' in buf['domains']: #Rv0697 and MftR and min
                    count = 14

            elif 'mycofact_glyco' in buf['domains']: #MftF and min
                count = 3
                if 'actino_HemFlav' in buf['domains']: #MftD and MftF and min
                    count = 12
                    if 'Rv0697' in buf['domains']: #Rv0697 and MftD and MftF and min
                        count = 18
                elif 'Rv0697' in buf['domains']: #Rv0697 and MftF and min
                    count = 15
            elif 'actino_HemFlav' in buf['domains']: #MftD and min
                count = 8
                if 'Rv0697' in buf['domains']: #MftD and Rv0697 and min
                    count = 10
            elif 'Rv0697' in buf['domains']: #Rv0697 and min
                count = 9
        coloring[mftb] = count
coloring = pd.DataFrame({'color': list(coloring.values())}, index = coloring.keys())

In [25]:
df_to_color = df_to_color.join(coloring)
df_to_color

,Description,List of IDs in Rep Node,name,Number of IDs in Rep Node,selected,Sequence,Sequence Length,Sequence Source,shared name,domains,...,mycofact_OYE_1,Fe-ADH,efflux_EmrB,Rxyl_3153,Response_reg~GerE,CoA_transf_3,Glyoxalase_6,Sig70_famx2,DNA_alkylation,color
seq_name,,,,,,,,,,,,,,,,,,,,,
WP_239101810.1,WP_239101810.1 mycofactocin radical SAM matura...,zzzz657,zzzz657,1,False,MTDPLDEAWELSPRVALRPEPFGALAYHFGNRRLTFLKRPELVSVL...,557,None,zzzz657,NaN,...,0,0,0,0,0,0,0,0,0,0
WP_228638331.1,WP_228638331.1 mycofactocin radical SAM matura...,zzzz574,zzzz574,1,False,MTNPLDEAWELSPSVALRPEPFGALAYHFGNRRLTFLKRRELVAVV...,529,None,zzzz574,"[mycofact_glyco, SDR_subfam_2, mycofact_creat,...",...,0,0,0,0,0,1,0,0,0,3
WP_189138050.1,WP_189138050.1 mycofactocin radical SAM matura...,zzzzz54,zzzzz54,1,False,MTSPLDQAWELSPSVALRPETFGAMAYHFGNRRLTFLKRTELVSVV...,525,None,zzzzz54,"[3a0107s01c2, 3a0107s02c, phosphate_pstC, ptsS...",...,0,0,0,1,0,0,0,0,0,0
WP_255280863.1,WP_255280863.1 mycofactocin radical SAM matura...,zzzz622,zzzz622,1,False,MARTPAASDFDLDAAWDLHPQVAVRPEPFGALLYHYGTRRLSFLKD...,523,None,zzzz622,NaN,...,0,0,0,0,0,0,0,0,0,0
WP_081878369.1,WP_081878369.1 mycofactocin radical SAM matura...,zzzz154,zzzz154,1,False,MFDLDDRWQLHPRVALRPEPFGALLYHFGTRKLSFLKSPRIVEIVR...,518,None,zzzz154,"[decarb_PcaC, DJ-1_PfpI, Aldo_ket_red, HTH_3, ...",...,0,0,0,0,0,0,0,0,1,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_257865691.1,WP_257865691.1 mycofactocin biosynthesis chape...,zzzz347,zzzz347,1,False,HFGTRKLSFLKNLTVVDIVKSLADHDSAEAALLAAGVTPAQRPLYV...,63,None,zzzz347,NaN,...,0,0,0,0,0,0,0,0,0,0
WP_214290027.1,WP_214290027.1 mycofactocin biosynthesis chape...,zzzz141,zzzz141,1,False,MWGLLTVPAPAQARRADSSEFDPDRGWRLHPQVAVRPEPFGALLYH...,58,None,zzzz141,"[Glyoxalase_6, DUF2332, mycofact_TetR, mycofac...",...,0,0,0,0,0,0,1,0,0,0
WP_119595456.1,WP_119595456.1 mycofactocin biosynthesis chape...,zzzz167,zzzz167,1,False,MSAMIDEAPAGFDWTRPWQLHPQVSLRPEPFGALLYHFGTRKLSFL...,58,None,zzzz167,"[mycofact_TetR, mycofactocin, Mycofactocin_RRE]",...,0,0,0,0,0,0,0,0,0,0


In [26]:
df_to_color.to_csv('/home/marfa/Desktop/lab/mycofactocin/mftb/cluster_res/new_to_cyt_mftb_cluster_rep_seq_node_with_operon_colored.csv')